In [1]:
import requests
from lxml import html
from pymongo import MongoClient
import time
import pandas as pd

In [2]:
url = 'https://www.worldometers.info/coronavirus/#main_table'

In [3]:
response = requests.get(url, headers = {
    'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36'})

In [4]:
tree = html.fromstring(response.content)

In [5]:
table_rows = tree.xpath("//table[@id='main_table_countries_today']/tbody/tr")
# ("//table[@class='table table-bordered table-hover main_table_countries dataTable no-footer'/tbody/tr[@class = 'odd']")
print(len(table_rows))

247


In [6]:
table_rows = table_rows[8:-8]
len(table_rows)

231

In [134]:
# Obtain every title of columns with tag <th>
headers = tree.xpath("//table[@id='main_table_countries_today']/thead/tr/th/text()")
print(headers)
print(len(headers))
print(type(headers[1]))

['#', 'Country,', 'Other', 'Total', 'Cases', 'New', 'Cases', 'Total', 'Deaths', 'New', 'Deaths', 'Total', 'Recovered', 'New', 'Recovered', 'Active', 'Cases', 'Serious,', 'Critical', 'Tot\xa0Cases/', '1M pop', 'Deaths/', '1M pop', 'Total', 'Tests', 'Tests/', '\n                                ', '\n                            ', 'Population', 'Continent', '1 Case', 'every X ppl', '1 Death', 'every X ppl', '1 Test', 'every X ppl', 'New Cases/1M pop', 'New Deaths/1M pop', 'Active Cases/1M pop']
39
<class 'lxml.etree._ElementUnicodeResult'>


In [135]:
headers = [str(h) for h in headers]
print(type(headers[1]))

<class 'str'>


In [136]:
for i in range(0,39):
    headers[i] = headers[i].replace(' ','')
    headers[i] = headers[i].replace('\xa0','')
print(headers)    

['#', 'Country,', 'Other', 'Total', 'Cases', 'New', 'Cases', 'Total', 'Deaths', 'New', 'Deaths', 'Total', 'Recovered', 'New', 'Recovered', 'Active', 'Cases', 'Serious,', 'Critical', 'TotCases/', '1Mpop', 'Deaths/', '1Mpop', 'Total', 'Tests', 'Tests/', '\n', '\n', 'Population', 'Continent', '1Case', 'everyXppl', '1Death', 'everyXppl', '1Test', 'everyXppl', 'NewCases/1Mpop', 'NewDeaths/1Mpop', 'ActiveCases/1Mpop']


In [137]:
headers[0]='Number'
headers[25]='Tests/1Mpop'
headers.pop(26)

print(headers)

['Number', 'Country,', 'Other', 'Total', 'Cases', 'New', 'Cases', 'Total', 'Deaths', 'New', 'Deaths', 'Total', 'Recovered', 'New', 'Recovered', 'Active', 'Cases', 'Serious,', 'Critical', 'TotCases/', '1Mpop', 'Deaths/', '1Mpop', 'Total', 'Tests', 'Tests/1Mpop', '\n', 'Population', 'Continent', '1Case', 'everyXppl', '1Death', 'everyXppl', '1Test', 'everyXppl', 'NewCases/1Mpop', 'NewDeaths/1Mpop', 'ActiveCases/1Mpop']


In [138]:
headers.pop(26)
print(headers)

['Number', 'Country,', 'Other', 'Total', 'Cases', 'New', 'Cases', 'Total', 'Deaths', 'New', 'Deaths', 'Total', 'Recovered', 'New', 'Recovered', 'Active', 'Cases', 'Serious,', 'Critical', 'TotCases/', '1Mpop', 'Deaths/', '1Mpop', 'Total', 'Tests', 'Tests/1Mpop', 'Population', 'Continent', '1Case', 'everyXppl', '1Death', 'everyXppl', '1Test', 'everyXppl', 'NewCases/1Mpop', 'NewDeaths/1Mpop', 'ActiveCases/1Mpop']


In [139]:
print(len(headers))

37


In [140]:
for h in range(1,12):
    headers[h]=headers[h]+headers[h+1]
    headers.pop(h+1)
print(headers)

['Number', 'Country,Other', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths', 'TotalRecovered', 'NewRecovered', 'ActiveCases', 'Serious,Critical', 'TotCases/1Mpop', 'Deaths/1Mpop', 'Total', 'Tests', 'Tests/1Mpop', 'Population', 'Continent', '1Case', 'everyXppl', '1Death', 'everyXppl', '1Test', 'everyXppl', 'NewCases/1Mpop', 'NewDeaths/1Mpop', 'ActiveCases/1Mpop']


In [141]:
headers[12]=headers[12]+headers[13]
headers.pop(13)
print(headers)

['Number', 'Country,Other', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths', 'TotalRecovered', 'NewRecovered', 'ActiveCases', 'Serious,Critical', 'TotCases/1Mpop', 'Deaths/1Mpop', 'TotalTests', 'Tests/1Mpop', 'Population', 'Continent', '1Case', 'everyXppl', '1Death', 'everyXppl', '1Test', 'everyXppl', 'NewCases/1Mpop', 'NewDeaths/1Mpop', 'ActiveCases/1Mpop']


In [142]:
headers[16]=headers[16]+headers[17]
headers.pop(17)
headers[17]=headers[17]+headers[18]
headers.pop(18)
headers[18]=headers[18]+headers[19]
headers.pop(19)
print(headers)

['Number', 'Country,Other', 'TotalCases', 'NewCases', 'TotalDeaths', 'NewDeaths', 'TotalRecovered', 'NewRecovered', 'ActiveCases', 'Serious,Critical', 'TotCases/1Mpop', 'Deaths/1Mpop', 'TotalTests', 'Tests/1Mpop', 'Population', 'Continent', '1CaseeveryXppl', '1DeatheveryXppl', '1TesteveryXppl', 'NewCases/1Mpop', 'NewDeaths/1Mpop', 'ActiveCases/1Mpop']


In [213]:
for row in table_rows:
    columns = row.xpath(".//td/text()")
#     columns = [str(col) for col in columns]
#     print(type(columns[0]))
    print(len(columns))
    print(columns)
    

16
['1', '109,668,403', '1,184,345                                ', '107,544,148', '939,910', '1,454', '327,559', '3,537', '1,186,431,916', '3,543,648', '                                ', 'North America', '3', '283', '0', '2,807']
17
['2', '45,002,561', '533,305                                ', 'N/A', 'N/A', 'N/A', 'N/A', '31,993', '379', '930,797,975', '661,721', '                                ', 'Asia', '31', '2,638', '2', '0.5']
13
['3', '40,138,560', '167,642                                ', '0', '612,013', '2,556', '271,490,188', '4,139,547', '                                ', 'Europe', '2', '391', '0']
20
['4', '38,678,277', '+6,566', '178,704                                ', '+87', '38,240,600', '258,973', 'N/A', '461,095', '2,130', '122,332,384', '1,458,359', '                                ', 'Europe', '2', '469', '1', '78', '1', '3,087']
16
['5', '38,078,411', '707,789                                ', '36,249,161', '1,121,461', 'N/A', '176,818', '3,287', '63,776,166

In [163]:
import lxml.html

def to_string(node):
    return lxml.html.tostring(node, encoding='unicode')

In [175]:
root = lxml.html.fromstring(columns)
text = root.cssselect('td')[0]

TypeError: expected string or bytes-like object, got 'list'

In [180]:
print(tree.xpath('.//td[contains(text(), "")]/text()'))

['\n                                    ', '\n                                ', '129,540,942', '1,656,614', '125,220,071', '+3,052', '2,664,257', '6,609', 'North America', '\n                                ', '\n                                    ', '\n                                ', '220,915,093', '+1', '1,552,068', '205,119,423', '+1,533', '14,243,602', '14,724', 'Asia', '\n                                ', '\n                                    ', '\n                                ', '251,642,829', '+6,566', '2,086,095', '+87', '245,505,178', '+3,550', '4,051,556', '4,666', 'Europe', '\n                                ', '\n                                    ', '\n                                ', '69,334,539', '1,363,348', '66,578,569', '1,392,622', '8,969', 'South America', '\n                                ', '\n                                    ', '\n                                ', '14,646,818', '31,029', '14,498,827', '116,962', '64', 'Australia/Oceania', '\n   

In [173]:
for td in tree:
    print(list(to_string(tree)))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [181]:

for row in table_rows:
    columns = row.xpath(".//td")
    cols=[]
#     print(len(columns))
#     print(columns)
#     сols = [('None' if td is None else td) for td in сolumns] 
    for td in columns:
        if not td:
            cols.append('empty')
        else:
            cols.append(td)
    print(len(cols))
    print(cols)

22
['empty', <Element td at 0x23bfcc13c50>, 'empty', 'empty', 'empty', 'empty', 'empty', 'empty', 'empty', 'empty', 'empty', 'empty', 'empty', 'empty', <Element td at 0x23bfcc10b90>, 'empty', 'empty', 'empty', 'empty', 'empty', 'empty', 'empty']
22
['empty', <Element td at 0x23bfcc10820>, 'empty', 'empty', 'empty', 'empty', 'empty', 'empty', 'empty', 'empty', 'empty', 'empty', 'empty', 'empty', <Element td at 0x23bfcc12080>, 'empty', 'empty', 'empty', 'empty', 'empty', 'empty', 'empty']
22
['empty', <Element td at 0x23bfcc10140>, 'empty', 'empty', 'empty', 'empty', <Element td at 0x23bfcc106e0>, 'empty', 'empty', 'empty', 'empty', 'empty', 'empty', 'empty', <Element td at 0x23bfcc109b0>, 'empty', 'empty', 'empty', 'empty', 'empty', 'empty', 'empty']
22
['empty', <Element td at 0x23bfcc10c80>, 'empty', 'empty', 'empty', 'empty', 'empty', 'empty', 'empty', 'empty', 'empty', 'empty', 'empty', 'empty', <Element td at 0x23bfcc100a0>, 'empty', 'empty', 'empty', 'empty', 'empty', 'empty', 'em

C:\Users\Yulia\AppData\Local\Temp\ipykernel_19216\2147922461.py:8: FutureWarning: The behavior of this method will change in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.
  if not td:


In [167]:
columns = tree.xpath("//table[@id='main_table_countries_today']/tbody/tr/td/text()")
print(type(columns))

<class 'list'>


In [141]:
def func_chunk(lst, n):
    for x in range(0, len(lst), n):
        e_c = lst[x : n + x]

        if len(e_c) < n:
            e_c = e_c + [None for y in range(n - len(e_c))]
        yield e_c

print(list(func_chunk(columns,22)))

[['\n                                    ', '\n                                ', '129,500,577', '1,656,458', '125,198,795', '+1,226', '2,645,324', '6,578', 'North America', '\n                                ', '\n                                    ', '\n                                ', '220,914,850', '1,552,064', '205,117,775', '+1,532', '14,245,011', '14,724', 'Asia', '\n                                ', '\n                                    ', '\n                                '], ['251,630,416', '+7,431', '2,085,880', '+15', '245,501,529', '+3,613', '4,043,007', '4,662', 'Europe', '\n                                ', '\n                                    ', '\n                                ', '69,334,539', '1,363,348', '66,578,569', '1,392,622', '8,969', 'South America', '\n                                ', '\n                                    ', '\n                                ', '14,646,818'], ['31,029', '14,498,827', '116,962', '64', 'Australia/Oceania', '\n    

In [158]:

for row in table_rows:
    columns = row.xpath(".//td")
    for td in columns:
        if not td :
            columns.append('<empty>')
        else:
            columns.append(td)
print(columns)

C:\Users\Yulia\AppData\Local\Temp\ipykernel_19216\3426339159.py:4: FutureWarning: The behavior of this method will change in future versions. Use specific 'len(elem)' or 'elem is not None' test instead.
  if not td :


KeyboardInterrupt: 

In [ ]:
for tr in soup.find_all('tr'):
    items = []

    for td in tr.find_all('td'):
        if not td.string:
            items.append('<empty>')
        else:
            items.append(td.string.strip())
            
    print(items)

In [211]:
data=[]
for row in table_rows:
    columns = row.xpath(".//td/text()")
    data.append({
        headers[0]:columns[0].strip(),
#         headers[1]:row.xpath(".//td/a[@class='mt_a']/text()")[0].strip(),
        headers[2]: columns[1].strip()
#          #  headers[3]:columns[4].strip(),
#         headers[4]:columns[2].strip(),
#         #  headers[5]:columns[6].strip(),
#         headers[6]:columns[3].strip(),
#         #  headers[7]:columns[6].strip(),
#         headers[8]: columns[4].strip(),
#         headers[9]: columns[5].strip(),
# #         if columns[6].strip() is not None:
# #             ' headers[10]':columns[6].strip()
# #         else:
# #             data.setdefault('total_cases/1M_pop'),
# #         headers[11]:columns[7].strip(),
# #         headers[12]:columns[8].strip(),
# #         headers[13]:columns[9].strip(),
#         headers[14]:row.xpath(".//td/a/text()")[1].strip()
# #          headers[15]
# #          headers[16]
# #          headers[17]
# #          headers[18]
# #          headers[19]
# #          headers[20]
# #          headers[21]
        })
print(data)

[{'Number': '1', 'TotalCases': '109,668,403'}, {'Number': '2', 'TotalCases': '45,002,561'}, {'Number': '3', 'TotalCases': '40,138,560'}, {'Number': '4', 'TotalCases': '38,678,277'}, {'Number': '5', 'TotalCases': '38,078,411'}, {'Number': '6', 'TotalCases': '34,571,873'}, {'Number': '7', 'TotalCases': '33,803,572'}, {'Number': '8', 'TotalCases': '26,415,845'}, {'Number': '9', 'TotalCases': '24,812,582'}, {'Number': '10', 'TotalCases': '23,430,629'}, {'Number': '11', 'TotalCases': '17,232,066'}, {'Number': '12', 'TotalCases': '13,914,811'}, {'Number': '13', 'TotalCases': '11,707,281'}, {'Number': '14', 'TotalCases': '11,624,114'}, {'Number': '15', 'TotalCases': '10,241,523'}, {'Number': '16', 'TotalCases': '10,080,046'}, {'Number': '17', 'TotalCases': '8,620,051'}, {'Number': '18', 'TotalCases': '7,649,199'}, {'Number': '19', 'TotalCases': '7,624,407'}, {'Number': '20', 'TotalCases': '6,814,248'}, {'Number': '21', 'TotalCases': '6,581,286'}, {'Number': '22', 'TotalCases': '6,384,891'}, {

In [164]:
for i in range(0,232):
    country = tree.xpath(".//td/a[@class='mt_a']/text()")[i].strip()
    print(country)


USA
India
France
Germany
Brazil
S. Korea
Japan
Italy
UK
Russia
Turkey
Spain
Australia
Vietnam
Taiwan
Argentina
Netherlands
Mexico
Iran
Indonesia
Poland
Colombia
Greece
Austria
Portugal
Ukraine
Chile
Malaysia
Israel
Belgium
Canada
DPRK
Thailand
Czechia
Peru
Switzerland
Philippines
South Africa
Romania
Denmark
Hong Kong
Sweden
Singapore
Serbia
New Zealand
Iraq
Hungary
Bangladesh
Slovakia
Georgia
Jordan
Ireland
Pakistan
Finland
Norway
Kazakhstan
Lithuania
Slovenia
Bulgaria
Guatemala
Morocco
Croatia
Lebanon
Costa Rica
Bolivia
Tunisia
Cuba
Ecuador
UAE
Panama
Uruguay
Mongolia
Nepal
Belarus
Latvia
Saudi Arabia
Azerbaijan
Paraguay
Bahrain
Sri Lanka
Dominican Republic
Kuwait
Cyprus
Myanmar
Moldova
Estonia
Palestine
Venezuela
Egypt
Qatar
Libya
Ethiopia
Réunion
Honduras
Armenia
Bosnia and Herzegovina
Oman
North Macedonia
Zambia
Kenya
Albania
Botswana
Luxembourg
Brunei
Montenegro
Algeria
Nigeria
Zimbabwe
Uzbekistan
Mozambique
Martinique
Afghanistan
Laos
Iceland
Kyrgyzstan
Guadeloupe
El Salvador
Tr

In [207]:
for i in columns:
    country = row.xpath(".//td/a[@class='mt_a']/text()")[0].strip()
    print(country)

USA


In [203]:
# //tbody/tr[@class='odd']/td[@style='font-weight: bold; text-align:right']/a/text()
for i in range(1,232,2):
    population = tree.xpath(".//td/a/text()")[i].strip()
    print(population)

334,805,269
1,406,631,776
65,584,518
83,883,596
215,353,593
51,329,899
125,584,838
60,262,770
68,497,907
145,805,947
85,561,976
46,719,142
26,068,792
98,953,541
23,888,595
46,010,234
17,211,447
131,562,772
86,022,837
279,134,505
37,739,785
51,512,762
10,316,637
9,066,710
10,140,570
43,192,122
19,250,195
33,181,072
9,326,000
11,668,278
38,388,419
25,990,679
70,078,203
10,736,784
33,684,208
8,773,637
112,508,994
60,756,135
19,031,335
5,834,950
7,604,299
10,218,971
5,943,546
8,653,016
4,898,203
42,164,965
9,606,259
167,885,689
5,460,193
3,968,738
10,300,869
5,020,199
229,488,994
5,554,960
5,511,370
19,205,043
2,661,708
2,078,034
6,844,597
18,584,039
37,772,756
4,059,286
6,684,849
5,182,354
11,992,656
12,046,656
11,305,652
18,113,361
10,081,785
4,446,964
3,496,016
3,378,078
30,225,582
9,432,800
1,848,837
35,844,909
10,300,205
7,305,843
1,783,983
21,575,842
11,056,370
4,380,326
1,223,387
55,227,143
4,013,171
1,321,910
5,345,541
29,266,991
106,156,692
2,979,915
7,040,745
120,812,698
908,061


In [193]:
population = tree.xpath(".//td/a/href[contains(@text, 'world-population')]/text()").strip()
print(population)

AttributeError: 'list' object has no attribute 'strip'

In [ ]:
//tbody/tr[@class='odd']/td[@style='font-weight: bold; text-align:right']/a/text()
row.xpath(".//td[@style='font-weight: bold; text-align:right']/a/text()")